# Install library

In [ ]:
!pip install tensorflow==2.15.0
!pip install tf-models-official==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.9 MB/s eta 

# Import library

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras import mixed_precision
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import tensorflow_models as tfm

2024-07-18 05:11:26.906637: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 05:11:26.906766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 05:11:27.047420: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Define hyperparammeter

In [ ]:
epochs = 4
batch_size = 16
init_lr = 2e-5

# Set use 16 bit

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


# Sentiment

## Load data

### Read csv

In [ ]:
df_train = pd.read_csv("/kaggle/input/synthetic_train.csv")
df_test = pd.read_csv("/kaggle/input/synthetic_val.csv")

### Review data
- Dataset have three column sentence and sentiment. We will use sentence as input and sentiment is output.
- We can see that this data is belance because all classes have same number of samples.
- Then, we count number words in each sentence. We can see that max word is 48. We don't need use large input size.

In [ ]:
df_train.head()

,sentence,sentiment,topic
0,Đội ngũ bảo trì quá thưa thớt dẫn đến không đả...,negative,facility
1,The university's musical and artistic faciliti...,neutral,facility
2,Phương pháp giảng dạy phù hợp với các đối tượn...,neutral,curriculum
3,Chương trình học giúp tôi trở thành một chuyên...,positive,curriculum
4,Tôi nghĩ rằng chương trình đào tạo có thể có t...,neutral,curriculum


In [ ]:
df_test.head()

,sentence,sentiment,topic
0,Chất lượng vật chất kém.,negative,facility
1,"Phần mềm học tập quá khó sử dụng, khiến sinh v...",negative,facility
2,Trường tôi thiếu những tiện ích cơ bản như máy...,negative,facility
3,Cần tạo thêm các hoạt động gắn kết giữa sinh v...,neutral,curriculum
4,Họ rất khoan dung và lượng giác trong quan điể...,neutral,others


In [ ]:
df_train.sentiment.value_counts()

sentiment
neutral     2724
negative    2711
positive    2709
Name: count, dtype: int64

In [ ]:
df_test.sentiment.value_counts()

sentiment
negative    686
positive    680
neutral     670
Name: count, dtype: int64

In [ ]:
df_train['len'] = df_train.sentence.apply(lambda x: len(str(x).split()))
df_test['len'] = df_test.sentence.apply(lambda x: len(str(x).split()))

In [ ]:
df_train['len'].describe()

count    8144.000000
mean       15.549730
std         5.018764
min         3.000000
25%        12.000000
50%        15.000000
75%        18.000000
max        43.000000
Name: len, dtype: float64

In [ ]:
df_test['len'].describe()

count    2036.000000
mean       15.694990
std         5.185957
min         2.000000
25%        12.000000
50%        15.000000
75%        19.000000
max        48.000000
Name: len, dtype: float64

## Create Dataset

### Create tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base', return_tensors="tf")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

### Create Dataset
Create train_data_generator and test_data_generator to generate data, just for sample in input and label and return this sample.

Use tf.data.Dataset.from_generator to create Dataset, we need set output_signature.

In [ ]:
train_texts = df_train.sentence.to_list()
test_texts = df_test.sentence.to_list()

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [ ]:
train_labels = df_train.sentiment.apply(lambda x: 0 if x == 'negative' else 1 if x == 'positive' else 2).to_list()
test_labels = df_test.sentiment.apply(lambda x: 0 if x == 'negative' else 1 if x == 'positive' else 2).to_list()

In [ ]:
def train_data_generator():
    for ids, mask, label in zip(train_encodings.input_ids, train_encodings.attention_mask, train_labels):
        yield (ids, mask), label

def test_data_generator():
    for ids, mask, label in zip(test_encodings.input_ids, test_encodings.attention_mask, test_labels):
        yield (ids, mask), label

# Create the dataset from the generator
train_dataset = tf.data.Dataset.from_generator(
    train_data_generator,
    output_signature=(
        (tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(None,), dtype=tf.int32)),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

test_dataset = tf.data.Dataset.from_generator(
    test_data_generator,
    output_signature=(
        (tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(None,), dtype=tf.int32)),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

## Create model

### Define model

In [ ]:
class custom_model(tf.keras.Model):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = TFAutoModel.from_pretrained('intfloat/multilingual-e5-base')
        self.dropout = layers.Dropout(0.3)
        self.attention = AttnHead(768, 768) #384 for small
        self.output_layer = layers.Dense(num_classes)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        x = self.base_model(input_ids, attention_mask).last_hidden_state
        x = self.attention(x)
        x = self.dropout(x)
        logits = self.output_layer(x)
        return logits

class AttnHead(tf.keras.layers.Layer):
    def __init__(self, hidden_dim, output_dim):
        super(AttnHead, self).__init__()
        self.linear_1 = layers.Dense(output_dim, activation='linear')
        self.linear_2 = layers.Dense(1, activation='linear')

    def call(self, last_hidden_states):
        linear_1_output = self.linear_1(last_hidden_states)
        activation = tf.nn.tanh(linear_1_output)
        score = self.linear_2(activation)
        weights = tf.nn.softmax(score, axis=1)
        output = tf.reduce_sum(weights * last_hidden_states, axis=1)

        return output

### Create learning rate scheduler

In [ ]:
# Number of warmup steps
steps_per_epoch = len(df_train) // batch_size
total_steps = steps_per_epoch * epochs
num_warmup_steps = min(500, int(0.1 * total_steps))

print(num_warmup_steps, total_steps)

# lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate=0,
#     decay_steps=total_steps - num_warmup_steps + 3,
#     alpha=0.0,
#     warmup_target=2e-5,
#     warmup_steps=num_warmup_steps
# )

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=init_lr,
    end_learning_rate=0,
    decay_steps=total_steps)

warmup_schedule = tfm.optimization.lr_schedule.LinearWarmup(
    warmup_learning_rate = 0,
    after_warmup_lr_sched = linear_decay,
    warmup_steps = num_warmup_steps
)

203 2036


### Create checkpoint callback

In [ ]:
checkpoint_filepath = '/kaggle/working/sentiment_base'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

### Create model

In [ ]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
  model = custom_model(3)
  model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=warmup_schedule, weight_decay=1e-2),
    #optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

Number of devices: 1


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


## Train model

In [ ]:
# Train the model
model.fit(
    train_dataset.shuffle(len(df_train)).batch(batch_size),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=test_dataset.batch(batch_size),
    callbacks=[model_checkpoint_callback]
)

Epoch 1/4
      1/Unknown - 72s 72s/step - loss: 2.0625 - accuracy: 0.1250

I0000 00:00:1721279596.179822      99 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    509/Unknown - 177s 207ms/step - loss: 0.5237 - accuracy: 0.7565
Epoch 1: val_accuracy improved from -inf to 0.83743, saving model to /kaggle/working/sentiment_base
509/509 [==============================] - 297s 444ms/step - loss: 0.5237 - accuracy: 0.7565 - val_loss: 0.3539 - val_accuracy: 0.8374
Epoch 2/4
509/509 [==============================] - ETA: 0s - loss: 0.2783 - accuracy: 0.8776
Epoch 2: val_accuracy improved from 0.83743 to 0.87967, saving model to /kaggle/working/sentiment_base
509/509 [==============================] - 212s 411ms/step - loss: 0.2783 - accuracy: 0.8776 - val_loss: 0.2725 - val_accuracy: 0.8797
Epoch 3/4
509/509 [==============================] - ETA: 0s - loss: 0.1895 - accuracy: 0.9170
Epoch 3: val_accuracy improved from 0.87967 to 0.88261, saving model to /kaggle/working/sentiment_base
509/509 [==============================] - 203s 394ms/step - loss: 0.1895 - accuracy: 0.9170 - val_loss: 0.2903 - val_accuracy: 0.8826
Epoch 4/4
509/509 [============

## Eval

In [ ]:
model.load_weights(checkpoint_filepath)

prediction = model.predict(test_dataset.batch(batch_size))
predict = prediction.argmax(-1).reshape(-1)

print(sklearn.metrics.classification_report(test_labels, predict, digits = 4))

128/128 [==============================] - 26s 56ms/step
              precision    recall  f1-score   support

           0     0.9868    0.9840    0.9854       686
           1     0.8633    0.8544    0.8588       680
           2     0.8409    0.8522    0.8466       670

    accuracy                         0.8973      2036
   macro avg     0.8970    0.8969    0.8969      2036
weighted avg     0.8976    0.8973    0.8974      2036



# Topic

## Review data
Dataset is balanced

In [ ]:
df_train.topic.value_counts()

topic
others        2059
curriculum    2040
lecturer      2026
facility      2019
Name: count, dtype: int64

In [ ]:
df_test.topic.value_counts()

topic
facility      526
lecturer      515
curriculum    507
others        488
Name: count, dtype: int64

## Create Dataset

In [ ]:
TOPIC_MAPPER = {'facility': 0,
               'lecturer': 1,
               'curriculum': 2,
               'others': 3}

In [ ]:
train_texts = df_train.sentence.to_list()
test_texts = df_test.sentence.to_list()

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [ ]:
train_labels = df_train.topic.map(TOPIC_MAPPER).to_list()
test_labels = df_test.topic.map(TOPIC_MAPPER).to_list()

In [ ]:
def train_data_generator():
    for ids, mask, label in zip(train_encodings.input_ids, train_encodings.attention_mask, train_labels):
        yield (ids, mask), label

def test_data_generator():
    for ids, mask, label in zip(test_encodings.input_ids, test_encodings.attention_mask, test_labels):
        yield (ids, mask), label

# Create the dataset from the generator
train_dataset = tf.data.Dataset.from_generator(
    train_data_generator,
    output_signature=(
        (tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(None,), dtype=tf.int32)),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

test_dataset = tf.data.Dataset.from_generator(
    test_data_generator,
    output_signature=(
        (tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(None,), dtype=tf.int32)),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

## Create model

### Create learning rate scheduler

In [ ]:
# Number of warmup steps
steps_per_epoch = len(df_train) // batch_size
total_steps = steps_per_epoch * epochs
num_warmup_steps = min(500, int(0.1 * total_steps))

print(num_warmup_steps, total_steps)

# lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate=0,
#     decay_steps=total_steps - num_warmup_steps + 3,
#     alpha=0.0,
#     warmup_target=2e-5,
#     warmup_steps=num_warmup_steps
# )

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=init_lr,
    end_learning_rate=0,
    decay_steps=total_steps)

warmup_schedule = tfm.optimization.lr_schedule.LinearWarmup(
    warmup_learning_rate = 0,
    after_warmup_lr_sched = linear_decay,
    warmup_steps = num_warmup_steps
)

203 2036


### Create checkpoint callback

In [ ]:
checkpoint_filepath = '/kaggle/working/topic_base'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

### Create model

In [ ]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
  model = custom_model(4)
  model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=warmup_schedule, weight_decay=1e-2),
    #optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

Number of devices: 1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


## Train model

In [ ]:
# Train the model
model.fit(
    train_dataset.shuffle(len(df_train)).batch(batch_size),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=test_dataset.batch(batch_size),
    callbacks=[model_checkpoint_callback]
)

Epoch 1/4
    509/Unknown - 161s 208ms/step - loss: 0.4007 - accuracy: 0.8460
Epoch 1: val_accuracy improved from -inf to 0.95334, saving model to /kaggle/working/topic_base
509/509 [==============================] - 285s 453ms/step - loss: 0.4007 - accuracy: 0.8460 - val_loss: 0.1273 - val_accuracy: 0.9533
Epoch 2/4
509/509 [==============================] - ETA: 0s - loss: 0.1000 - accuracy: 0.9671
Epoch 2: val_accuracy improved from 0.95334 to 0.95874, saving model to /kaggle/working/topic_base
509/509 [==============================] - 199s 387ms/step - loss: 0.1000 - accuracy: 0.9671 - val_loss: 0.1307 - val_accuracy: 0.9587
Epoch 3/4
509/509 [==============================] - ETA: 0s - loss: 0.0432 - accuracy: 0.9848
Epoch 3: val_accuracy improved from 0.95874 to 0.96365, saving model to /kaggle/working/topic_base
509/509 [==============================] - 215s 418ms/step - loss: 0.0432 - accuracy: 0.9848 - val_loss: 0.1344 - val_accuracy: 0.9637
Epoch 4/4
509/509 [==============

## Eval

In [ ]:
model.load_weights(checkpoint_filepath)

prediction = model.predict(test_dataset.batch(batch_size))
predict = prediction.argmax(-1).reshape(-1)

print(sklearn.metrics.classification_report(test_labels, predict, digits = 4))

128/128 [==============================] - 18s 57ms/step
              precision    recall  f1-score   support

           0     0.9579    0.9506    0.9542       526
           1     0.9786    0.9748    0.9767       515
           2     0.9333    0.9389    0.9361       507
           3     0.9898    0.9959    0.9928       488

    accuracy                         0.9646      2036
   macro avg     0.9649    0.9650    0.9649      2036
weighted avg     0.9646    0.9646    0.9646      2036



# Multi Output

## Create Dataset
In data_generator, we need return dictional for label, because we have two output (tensorflow auto use signature output_1 and output_2), than we return {'output_1': sentiment, 'output_2': topic}.

If you don't want to use output_1, output_2, you can set {'sentiment': sentiment, 'topic': topic}. But you need change:
- callback: output_1_accuracy --> sentiment_accuracy, output_2_accuracy --> topic_accuracy
- model: return sentiment, topic --> return {'sentiment': sentiment, 'topic': topic}
- tf.data.Dataset.from_generator --> output_signature: output_1 --> sentiment, output_2 --> topic
- model.compile --> metrics = {'output_1': 'accuracy', 'output_2': 'accuracy'} --> metrics = {'sentiment': 'accuracy', 'topic': 'accuracy'}
- Eval:
  - predict_sentiment = model.predict(test_dataset.batch(batch_size))[0] --> predict_sentiment = model.predict(test_dataset.batch(batch_size))['sentiment']
  - predict_sentiment = model.predict(test_dataset.batch(batch_size))[1] --> predict_sentiment = model.predict(test_dataset.batch(batch_size))['topic']

In [ ]:
train_texts = df_train.sentence.to_list()
test_texts = df_test.sentence.to_list()

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [ ]:
train_sentiment_labels = df_train.sentiment.apply(lambda x: 0 if x == 'negative' else 1 if x == 'positive' else 2).to_list()
test_sentiment_labels = df_test.sentiment.apply(lambda x: 0 if x == 'negative' else 1 if x == 'positive' else 2).to_list()

In [ ]:
train_topic_labels = df_train.topic.map(TOPIC_MAPPER).to_list()
test_topic_labels = df_test.topic.map(TOPIC_MAPPER).to_list()

In [ ]:
def train_data_generator():
    for ids, mask, label_sentiment, label_topic in zip(train_encodings.input_ids,
                                                       train_encodings.attention_mask,
                                                       train_sentiment_labels,
                                                       train_topic_labels):
        yield (ids, mask), {'output_1': label_sentiment, 'output_2': label_topic}

def test_data_generator():
    for ids, mask, label_sentiment, label_topic in zip(test_encodings.input_ids,
                                test_encodings.attention_mask,
                                test_sentiment_labels,
                                test_topic_labels):
        yield (ids, mask), {'output_1': label_sentiment, 'output_2': label_topic}

# Create the dataset from the generator
train_dataset = tf.data.Dataset.from_generator(
    train_data_generator,
    output_signature=(
        (tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(None,), dtype=tf.int32)),
        {'output_1': tf.TensorSpec(shape=(), dtype=tf.int32),
         'output_2': tf.TensorSpec(shape=(), dtype=tf.int32)}
    )
)

test_dataset = tf.data.Dataset.from_generator(
    test_data_generator,
    output_signature=(
        (tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(None,), dtype=tf.int32)),
        {'output_1': tf.TensorSpec(shape=(), dtype=tf.int32),
         'output_2': tf.TensorSpec(shape=(), dtype=tf.int32)}
    )
)

## Create model

### Create learning rate scheduler

In [ ]:
# Number of warmup steps
steps_per_epoch = len(df_train) // batch_size
total_steps = steps_per_epoch * epochs
num_warmup_steps = min(500, int(0.1 * total_steps))

print(num_warmup_steps, total_steps)

# lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate=0,
#     decay_steps=total_steps - num_warmup_steps + 3,
#     alpha=0.0,
#     warmup_target=2e-5,
#     warmup_steps=num_warmup_steps
# )

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=init_lr,
    end_learning_rate=0,
    decay_steps=total_steps)

warmup_schedule = tfm.optimization.lr_schedule.LinearWarmup(
    warmup_learning_rate = 0,
    after_warmup_lr_sched = linear_decay,
    warmup_steps = num_warmup_steps
)

203 2036


### Create checkpoint callback

In [ ]:
checkpoint_sentiment_filepath = '/kaggle/working/model_sentiment'
model_checkpoint_sentiment_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_sentiment_filepath,
    save_best_only=True,
    monitor='val_output_1_accuracy',
    mode='max',
    verbose=1
)

checkpoint_topic_filepath = '/kaggle/working/model_topic'
model_checkpoint_topic_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_topic_filepath,
    save_best_only=True,
    monitor='val_output_2_accuracy',
    mode='max',
    verbose=1
)

### Create model
In model.compile, we need set loss, loss_weights and metrics for each output.

In [ ]:
class custom_model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.base_model = TFAutoModel.from_pretrained('intfloat/multilingual-e5-base')
        self.dropout = layers.Dropout(0.3)
        self.attention = AttnHead(768, 768) #384 for small
        self.sentiment = layers.Dense(3, name = 'sentiment')
        self.topic = layers.Dense(4, name = 'topic')

    def call(self, inputs):
        input_ids, attention_mask = inputs
        x = self.base_model(input_ids, attention_mask).last_hidden_state
        x = self.attention(x)
        x = self.dropout(x)
        sentiment = self.sentiment(x)
        topic = self.topic(x)
        return sentiment, topic

class AttnHead(tf.keras.layers.Layer):
    def __init__(self, hidden_dim, output_dim):
        super(AttnHead, self).__init__()
        self.linear_1 = layers.Dense(output_dim, activation='linear')
        self.linear_2 = layers.Dense(1, activation='linear')

    def call(self, last_hidden_states):
        linear_1_output = self.linear_1(last_hidden_states)
        activation = tf.nn.tanh(linear_1_output)
        score = self.linear_2(activation)
        weights = tf.nn.softmax(score, axis=1)
        output = tf.reduce_sum(weights * last_hidden_states, axis=1)

        return output

In [ ]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
  model = custom_model()
  model.compile(
    optimizer = tf.keras.optimizers.AdamW(learning_rate=warmup_schedule, weight_decay=1e-2),
    #optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss = [tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
           ],
    loss_weights = [1, 0.5],
    metrics = {'output_1': 'accuracy', 'output_2': 'accuracy'}
)

Number of devices: 1


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


## Train model

In [ ]:
# Train the model
model.fit(
    train_dataset.shuffle(len(df_train)).batch(batch_size),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=test_dataset.batch(batch_size),
    callbacks=[model_checkpoint_sentiment_callback, model_checkpoint_topic_callback]
)

Epoch 1/4
    509/Unknown - 162s 182ms/step - loss: 0.7939 - output_1_loss: 0.5552 - output_2_loss: 0.4774 - output_1_accuracy: 0.7419 - output_2_accuracy: 0.8088
Epoch 1: val_output_1_accuracy improved from -inf to 0.85756, saving model to /kaggle/working/model_sentiment

Epoch 1: val_output_2_accuracy improved from -inf to 0.94941, saving model to /kaggle/working/model_topic
509/509 [==============================] - 392s 635ms/step - loss: 0.7939 - output_1_loss: 0.5552 - output_2_loss: 0.4774 - output_1_accuracy: 0.7419 - output_2_accuracy: 0.8088 - val_loss: 0.3871 - val_output_1_loss: 0.3162 - val_output_2_loss: 0.1418 - val_output_1_accuracy: 0.8576 - val_output_2_accuracy: 0.9494
Epoch 2/4
509/509 [==============================] - ETA: 0s - loss: 0.3441 - output_1_loss: 0.2842 - output_2_loss: 0.1198 - output_1_accuracy: 0.8697 - output_2_accuracy: 0.9628
Epoch 2: val_output_1_accuracy improved from 0.85756 to 0.87230, saving model to /kaggle/working/model_sentiment

Epoch 2: 

## Eval
Because model return two outputs (return sentiment, topic). function model.predict will return tupple with first element is sentiment predict tensor and second element is topic predict tensor

If use return {'sentiment': sentiment, 'topic': topic} in model, model.predict will return dictionary {'sentiment': sentiment predict tensor, 'topic': topic topic tensor}, then you need replace [0] with ['sentiment'] and [1] with ['topic']

In [ ]:
model.load_weights(checkpoint_sentiment_filepath)

predict_sentiment = model.predict(test_dataset.batch(batch_size))[0]
predict_sentiment = predict_sentiment.argmax(-1).reshape(-1)

print(sklearn.metrics.classification_report(test_sentiment_labels, predict_sentiment, digits = 4))

128/128 [==============================] - 18s 57ms/step
              precision    recall  f1-score   support

           0     0.9840    0.9840    0.9840       686
           1     0.8625    0.8765    0.8694       680
           2     0.8604    0.8463    0.8533       670

    accuracy                         0.9028      2036
   macro avg     0.9023    0.9022    0.9022      2036
weighted avg     0.9027    0.9028    0.9027      2036



In [ ]:
model.load_weights(checkpoint_topic_filepath)

predict_topic = model.predict(test_dataset.batch(batch_size))[1]
predict_topic = predict_topic.argmax(-1).reshape(-1)

print(sklearn.metrics.classification_report(test_topic_labels, predict_topic, digits = 4))

128/128 [==============================] - 7s 57ms/step
              precision    recall  f1-score   support

           0     0.9308    0.9715    0.9507       526
           1     0.9748    0.9748    0.9748       515
           2     0.9545    0.9112    0.9324       507
           3     0.9877    0.9877    0.9877       488

    accuracy                         0.9612      2036
   macro avg     0.9619    0.9613    0.9614      2036
weighted avg     0.9615    0.9612    0.9611      2036

